In [23]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from rps_rllib import RPSNoise
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.examples.env.rock_paper_scissors import RockPaperScissors
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.registry import register_env, _global_registry, ENV_CREATOR
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy

import ray
from ray import tune
from ray.tune import track

import math

In [41]:
def ray_init():
    ray.shutdown()
    return ray.init(ignore_reinit_error=True, include_webui=True,
                    temp_dir='/scratch/sergei/tmp') # Skip or set to ignore if already called

In [42]:
ray_init()

2020-07-04 16:51:12,851	INFO resource_spec.py:212 -- Starting Ray with 54.2 GiB memory available for workers and up to 27.1 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-04 16:51:13,131	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-07-04 16:51:13,390	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.32.175.9',
 'raylet_ip_address': '128.32.175.9',
 'redis_address': '128.32.175.9:6379',
 'object_store_address': '/scratch/sergei/tmp/session_2020-07-04_16-51-12_849122_13290/sockets/plasma_store',
 'raylet_socket_name': '/scratch/sergei/tmp/session_2020-07-04_16-51-12_849122_13290/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/scratch/sergei/tmp/session_2020-07-04_16-51-12_849122_13290'}

In [3]:
env_config = {}
env_cls = RockPaperScissors
#register_env("RPSNoise", lambda c: env_cls(c))

In [31]:
def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "learned%02d"

    agent_config = (PPOTFPolicy, obs_space, act_space, {
                    "model": {
                        "use_lstm": True,
                        "fcnet_hiddens": [config['fc_units'], config['fc_units']],
                        "lstm_cell_size": config['lstm_units'],
                    },
                    "framework": "tfe",
                })

    N_POLICIES = 2

    policies = {policy_template % i: agent_config for i in range(N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player1", "player2"]
        agent_ids = ["player1", "player2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_cls,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
    #   "train_batch_size": 200,
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    config = build_trainer_config(restore_state=restore_state,
                                  train_policies=train_policies,
                                  config=config)
    
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

def train(trainer, stop_iters):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        track(results)
    o = trainer.save_to_object()
    return o

In [32]:
node_sizes = [math.ceil(t) for t in np.logspace(0, 3, 10)]

In [37]:
config = build_trainer_config(config={'fc_units': tune.uniform(node_sizes),
                             'lstm_units': tune.uniform(node_sizes)})
config['train_steps'] = 100

In [38]:
def train_one(config):
    trainer = build_trainer(restore_state=None, train_policies=['learned00'], config=config)
    train(trainer, config['train_steps'])

In [ ]:
ray_init()

analysis = tune.run(
    train_one, 
    config=config, 
    verbose=1,
    name="train_mnist",  # This is used to specify the logging directory.
    stop={"mean_accuracy": 0.98}  # This will stop the trial 
)

In [ ]:
train(trainer, 1000)

In [ ]:
trainer.config

In [ ]:
# for alternating training
joint_steps = 10
opponent_steps = 10
burst_size = 10

In [ ]:
dir(trainer)

In [ ]:
?trainer.export_model

In [ ]:
trainer.save()

In [ ]:
trainer.load()

In [ ]:
trainer.restore()

In [ ]:
dir(trainer)

In [ ]:
len(o)

In [ ]:
trainer.config

In [ ]:
dir(trainer)

In [ ]:
trainer.with_updates(multiagent={'policies_to_train': ['learned00']})

In [ ]:
?trainer.with_updates